In [121]:
import psycopg2
import pandas as pd
from matplotlib.cbook import dedent
from tqdm import tqdm

In [2]:
dbname = 'podcast'
username = 'lindsay'

con = psycopg2.connect(database = dbname, user = username)
cursor = con.cursor()

In [14]:
# select 25 random podcasts with at least 5 'also-subscribed'

query = """
SELECT initial_podcast, count(subscribed_podcast) 
FROM also_subscribed 
GROUP BY initial_podcast 
HAVING count(subscribed_podcast) > 5 
ORDER BY random() LIMIT 25;
"""
query = query.replace('\n', ' ')
cursor.execute(query)
query_results = cursor.fetchall()


In [16]:
test_podcasts = pd.DataFrame({'id' : [x[0] for x in query_results]})

In [122]:
subscribed_query = """
SELECT initial_podcast, subscribed_podcast, similarity
FROM also_subscribed
INNER JOIN podcast_similarity ON also_subscribed.initial_podcast=podcast_similarity.search_id
AND also_subscribed.subscribed_podcast=podcast_similarity.match_id
WHERE initial_podcast=%s;
"""
subscribed_query = subscribed_query.replace('\n', ' ')

unsubscribed_begin = """
SELECT search_id, match_id, similarity
FROM podcast
INNER JOIN podcast_similarity
ON podcast.id=podcast_similarity.search_id
WHERE search_id=%s
AND match_id NOT IN 
"""
unsubscribed_end = """
ORDER BY random() LIMIT %s
"""


In [ ]:
also_subscribed_similarity_df = pd.DataFrame(columns=['search_id', 'match_id', 'subscribe_status', 'similarity'])
num_unsubscribed = 25
for ind, podcast in tqdm(test_podcasts.iterrows(), total=test_podcasts.shape[0]):
    
    # get subscribed podcasts
    podcast_id = podcast['id']
    data = (test_podcasts.iloc[0]['id'], )
    cursor.execute(subscribed_query, data)
    query_results = cursor.fetchall()
    subscribed = [x[1] for x in query_results]
    sim_scores = [float(x[2]) for x in query_results]
    this_result = pd.DataFrame({'search_id' : [podcast_id] * len(query_results),
                                'match_id' : subscribed,
                                'subscribe_status' : [True] * len(query_results),
                                'similarity' : sim_scores})
    also_subscribed_similarity_df.append(this_result)
    
    #### get unsubscribed podcasts
    
    # build query
    unsubscribed_middle = '('
    for i in range(len(subscribed)):
        if i != len(subscribed) - 1:
            unsubscribed_middle = unsubscribed_middle + '%s::bigint, '
        else:
            unsubscribed_middle = unsubscribed_middle + '%s::bigint)'

    unsubscribed_query = unsubscribed_begin + unsubscribed_middle + unsubscribed_end
    unsubscribed_query = unsubscribed_query.replace('\n', ' ')
    
    # build data
    data = (podcast_id,)
    for i in range(len(subscribed)):
        data = data + (subscribed[i],)
    data = data + (num_unsubscribed, )
    
    # execute
    cursor.execute(unsubscribed_query, data)
    query_results = cursor.fetchall()
    sim_scores = [float(x[2]) for x in query_results]
    unsubscribed = [x[1] for x in query_results]
    this_result = pd.DataFrame({'search_id' : [podcast_id] * len(query_results),
                                'match_id' : unsubscribed_begin,
                                'subscribe_status' : [False] * len(query_results),
                                'similarity' : sim_scores})
    also_subscribed_similarity_df.append(this_result)

  8%|▊         | 2/25 [00:28<05:33, 14.48s/it]